In [25]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import folium
import matplotlib as mpl
import matplotlib.cm as cm
pd.options.display.max_columns = 200

In [44]:
#считываем данные
df1 = pd.read_csv('Rus_schools_final.csv', encoding = '1251')
df1.head()

,Unnamed: 0,name,struct,addr,lat,lon
0,0,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420087, Республика Татарстан, г. Казань, ул.Ри...",55.76370,49.18172
1,1,"Муниципальная образовательная школа-интернат ""...",(Муниципальное образовательное учреждение),"420103, Республика Татарстан, г. Казань, ул.Че...",55.82440,49.12312
2,2,Муниципальное учреждение образования для детей...,(Муниципальное образовательное учреждение),"420100, Республика Татарстан, г. Казань, ул. Ю...",55.74462,49.20560
3,3,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420011, Республика Татарстан, г. Казань, Ферма-2",55.71716,49.16222
4,4,Специальное (коррекционное)образовательное учр...,(Государственное образовательное учреждение),"420036, Республика Татарстан, г. Казань, ул.Ти...",55.84911,49.07660


In [46]:
# Моей школы нет в списке данных, добавляем в ручную
dict_lider = {'name': 'ГБОУ Волгоградский лицей-интернат "Лидер"', 'struct': '(Государственное образовательное учреждение)',
                            'addr' : '400011, Волгоградская область, Волгоград, просп. Университетский, 34', 'lat': 48.669294, 'lon': 44.439448}
df1 = pd.concat([df1, pd.DataFrame([dict_lider])], ignore_index = True)
df1.tail()

,Unnamed: 0,addr,lat,lon,name,struct
42577,42577.0,"301131, Тульская область, Ленинский район, п. ...",51.149130,49.583650,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение)
42578,42578.0,"301823, Тульская область, Богородицкий район, ...",53.681980,38.200980,Муниципальное бюджетное образовательное учрежд...,(Муниципальное образовательное учреждение)
42579,42579.0,"301300, Тульская область, Веневский район, пос...",54.351710,38.271900,"Муниципальное образовательное учреждение ""Морд...",(Муниципальное образовательное учреждение)
42580,42580.0,"301948, Тульская область, Куркинский район, п....",53.430790,38.655160,Муниципальное общеобразовательное учреждение С...,(Муниципальное образовательное учреждение)
42581,NaN,"400011, Волгоградская область, Волгоград, прос...",48.669294,44.439448,"ГБОУ Волгоградский лицей-интернат ""Лидер""",(Государственное образовательное учреждение)


In [51]:
#Выкидываем ненужный столбец
df1 = df1.drop(columns = ['Unnamed: 0'])

In [110]:
from math import radians, atan2, sin, cos, sqrt

Volgograd_schools = df1['addr'].str.split(',').str[1].str.contains('Волгоградская область').astype(bool).values
map_folium_school = folium.Map(location = (dict_lider['lat'], dict_lider['lon']), tiles = 'Stamen Toner', 
                        zoom_start = 11)

def distance_calculate(a, b): # betwen two points of template (lat, lon) in degrees
    """
    Рассчитывает расстояние между двумя точками на карте в км.
    a и b словари, которые имеют ключи 'lat', 'lon' используется формула ‘haversine’ 
    точность порядка 0.5 % для относительно близко расположенных точках. Если учитывать точки находящиеся на расстоянии порядка
    1 тыс. км нужно использовать другие, более сложные рассчёты. Так как мы работаем в пределах 1 тыс. км в Волгоградской области,
    то можно вполне пользоваться предложенным рассчётом.
    """
    R = 6371.0 # km
    assert(isinstance(a, dict))
    assert(isinstance(b, dict))
    lata = radians(a['lat'])
    latb = radians(b['lat'])
    lona, lonb = radians(a['lon']), radians(b['lon'])
    dlon = lonb - lona
    dlat = latb - lata
    a = sin(dlat / 2)**2 + cos(lata) * cos(latb) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance


def mask_radius_Volgograd(radius_in_km):
    """
    По переданному аргументу radius_in_km выдает список индексов тех школ области, что находяться
    в радиусе radius_in_km. 
    radius_in_km - в км
    """
    x_y_my = df1[['lat', 'lon']].iloc[-1].to_dict()
    assert(x_y_my['lat'] == dict_lider['lat'] and x_y_my['lon'] == dict_lider['lon'])
    Volgograd_schools = df1['addr'].str.split(',').str[1].str.contains('Волгоградская область').astype(bool).values
    Volgograd_schools[-1] = True 
    distance_new = df1[Volgograd_schools].apply(lambda r: distance_calculate(r.to_dict(), dict_lider), axis = 1)
    mask = df1[Volgograd_schools].apply(lambda r: distance_calculate(r.to_dict(), dict_lider) < radius_in_km, axis = 1)
    index_positive = mask[mask == True].index
    return index_positive

km = 10. # Можно менять и видеть разные результаты
for ind, row in df1.iloc[mask_radius_Volgograd(km)].iterrows():
    folium.CircleMarker([row['lat'], row['lon']], radius = 7,
                                fill = True, 
                                color = 'red' if row['name'] == dict_lider['name'] else 'blue',
                                fill_opacity = 0.9).add_to(map_folium_school)
    

folium.Circle(
            location = [dict_lider['lat'], dict_lider['lon']],
            radius = km * 1000,
            popup = 'Лицей Лидер',
            color = 'crimson',
            fill = False).add_to(map_folium_school)

map_folium_school.save('school_map.html')
map_folium_school